In [ ]:
from dlx import DLX
import logging
logging.getLogger().setLevel(logging.INFO)
import pdb

In [ ]:
logging.info("infoidsj")
logging.warn("waren")
logging.debug("dlsj")
logging.error("erjl")

In [ ]:
# Define simple tile coverage of 2x3 square
columns = [(i,0) for i in ["11","12","13","21","22",dict(a=5)]]
rows = (([1,1,0,1,0,0],"topleftL"),
        ([1,0,0,1,0,0],"leftside"),
        ([0,0,0,1,1,1],"bottom"),
        ([0,0,0,0,1,1],"bottomright"),
        ([0,0,1,0,0,0],"topright"),
        ([0,1,1,0,0,1],"toprightL"),
        ([1,1,1,1,1,1],"trivial"),
        ([0,0,0,0,0,0],"antitrivial"))
rows = (([0,1,3],"topleftL"),
        ([0,3],"leftside"),
        ([3,4,5],"bottom"),
        ([4,5],"bottomright"),
        ([2],"topright"),
        ([1,2,5],"toprightL"),
        ([0,1,2,3,4,5],"trivial"),
        )
rowshapes, rownames = zip(*rows)
rowshapes, rownames

In [ ]:
dl = DLX(columns)
dl.appendRows(rowshapes, rownames)

In [ ]:
for sol in dl.solve():
    dl.printSolution(sol)
    print

In [ ]:
dl.useRow(7)
for sol in dl.solve():
    dl.printSolution(sol)
    print

In [ ]:
DLX??

In [ ]:
dl.getRowList(10)

In [ ]:
columns = [('a',DLX.PRIMARY), ('b',DLX.PRIMARY), ('c',DLX.PRIMARY), ('d',DLX.SECONDARY), ([1,2,3],DLX.PRIMARY)]
d = DLX(columns)
rows = [[1,2,4],
        [0,1,3],
        [0],
        [0,1,2,3,4]]
rowNames = ['row%i' % i for i in range(len(rows))]
d.appendRows(rows, rowNames)
for sol in d.solve():
    d.printSolution(sol)
    print
list(d.solve())

In [ ]:
d.getRowList(7)

In [ ]:
class Group(object):
    def __init__(self,name):
        self.name = name
        self._cells = []
    def addCell(self,cell):
        self._cells.append(cell)
    def __repr__(self):
        return self.name
        return "Group(name=%s,%d cells)"%(self.name,len(self._cells))
    def __str__(self):
        return self.name
class Cell(object):
    def __init__(self,name=None,value=None):
        self._groups = []
        self._value = value
        self.name = name
    def addToGroup(self,group):
        logging.debug("Adding %s to %s",self, group)
        self._groups.append(group)
        group.addCell(self)
    def getGroups(self):
        return self._groups
    def setValue(self,value):
        self._value = value
    def getValue(self):
        return self._value
    def __repr__(self):
        return self.name
        return "Cell(name=%s,value=%s,%d groups)"%(self.name,self._value,len(self._groups))
    def __str__(self):
        return self.name
class LatinSquare(object):
    def __init__(self,n,groups=[]):
        self.n = n
        self._groups = groups
    def getGroups(self):
        return self._groups
    def __repr__(self):
        return "LatinSquare(n=%d,%d groups)"%(self.n,len(self._groups))
    def __str__(self):
        return "LatinSquare(%d)"%self.n
class Sudoku(LatinSquare):
    def __init__(self,groupwidth,groupheight=None,board=None):
        self._group_width = groupwidth
        self._group_height = groupheight or groupwidth
        # Numbers to fit 
        n = self._group_width*self._group_height
        
        self._rows = [Group("Row(%d)"%i) for i in xrange(n)]
        self._cols = [Group("Col(%d)"%i) for i in xrange(n)]
        self._blocks = []
        self._cells = [[Cell("Cell(%d,%d)"%(c,r)) for r in xrange(n)] for c in xrange(n)]
        self._values = range(1,n+1)
        
        for by in xrange(self._group_width):
            for bx in xrange(self._group_height):
                block = Group("Block(%d,%d)"%(bx,by))
                self._blocks.append(block)
                for x in xrange(self._group_width):
                    for y in xrange(self._group_height):
                        c = bx*self._group_width+x
                        r = by*self._group_height+y
                        logging.debug("Adding groups to %d,%d",c,r)
                        for g in [self._rows[r],self._cols[c],block]:
                            self._cells[c][r].addToGroup(g)
                        
        super(Sudoku,self).__init__(n, self._rows + self._cols + self._blocks)
        self.setBoard(board)
    def setValue(self,col, row, value):
        self[col][row].setValue(self._values[int(value)-1])
    def setBoard(self,board):
        if type(board) == str:
            lines = board.split("\n")
            if len(lines) != self.n:
                raise ValueError("Wrong number of rows. Expected %d, found %d."%(self.n,len(lines)))
            for r in xrange(self.n):
                for c in xrange(self.n):
                    if c >= len(lines[r]): break
                    val = lines[r][c] #invert normal x,y order
                    if val != " " and val != "_":
                        try:
                            val = int(lines[r][c])
                        except: pass
                        self.setValue(c,r,val)
    def getValues(self):
        return self._values
    def __len__(self):
        return self.n
    def __getitem__(self,c):
        """sudoku[col][row] -> Cell"""
        return self._cells[c]
    def __str__(self):
        return "\n".join([" ".join([str(self._cells[c][r].getValue() or "_") for c in xrange(self.n)]) for r in xrange(self.n)])
    
class Technique(object):
    pass
class DLXTechnique(Technique):
    def __init__(self,square):
        self._square = square
        
        groups = square.getGroups()
        values = square.getValues()

        constraints = [(g,v) for g in groups for v in values] #3n*n
        constraints += [square[c][r] for c in xrange(len(square)) for r in xrange(len(square))] #n*n
        
        
        blocks = []
        names = []
        
        for c in xrange(len(square)):
            for r in xrange(len(square)):
                cell = square[c][r]
                groups = cell.getGroups()
                for value in square.getValues():
                    names.append( (cell,value) )
                    blocks.append( [ constraints.index( (g,value) ) for g in groups] + [constraints.index(cell)])
        
        self.dlx = DLX(zip(constraints,[DLX.PRIMARY]*len(constraints)))
        self._rowNums = self.dlx.appendRows(blocks,names)
        self._blocks = blocks
        self._blockNames = names
        
        self.update()
        
    def update(self):
        for name,num in zip(self._blockNames,self._rowNums):
            cell,val = name
            if cell.getValue() == val:
                self.dlx.useRow(num)
            
    def solve(self):
        soln = self.dlx.solve()
        for rowNum in soln.next():
            cell,val = d.dlx.N[rowNum]
            cell.setValue(val)
            logging.debug("Solved %d at %s"%(val,cell))
    def solutions(self):
        for soln in self.dlx.solve():
            for rowNum in soln:
                cell,val = d.dlx.N[rowNum]
                cell.setValue(val)
            yield self._square
        

In [ ]:
s = Sudoku(2,2)
s.setBoard(""" 1  
  2 
   3
4   """)
print s

In [ ]:
s[1][0].getValue()

In [ ]:
d = DLXTechnique(s)
for ss in d.solutions():
    print ss
    print



In [ ]:
board="""   16 
     4
 2   3
3   5 
4     
 51   """
s = Sudoku(3,2,board)
print s

In [ ]:
d = DLXTechnique(s)
for ss in d.solutions():
    print ss
    print

In [ ]:
print "\n".join([" ".join(["%s-%s"%(s[c][r].getGroups()[2],s[c][r].getValue()) for c in range(len(s))]) for r in range(len(s))])

In [63]:
board="""___6_47__
7_6_____9
_____5_8_
_7__2__93
8_______5
43__1__7_
_5_2_____
3_____2_8
__23_1___"""
s = Sudoku(3,3,board)
print s

_ _ _ 6 _ 4 7 _ _
7 _ 6 _ _ _ _ _ 9
_ _ _ _ _ 5 _ 8 _
_ 7 _ _ 2 _ _ 9 3
8 _ _ _ _ _ _ _ 5
4 3 _ _ 1 _ _ 7 _
_ 5 _ 2 _ _ _ _ _
3 _ _ _ _ _ 2 _ 8
_ _ 2 3 _ 1 _ _ _


In [64]:
d = DLXTechnique(s)
for ss in d.solutions():
    print ss
    print

5 8 3 6 9 4 7 2 1
7 1 6 8 3 2 5 4 9
2 9 4 1 7 5 3 8 6
6 7 1 5 2 8 4 9 3
8 2 9 7 4 3 1 6 5
4 3 5 9 1 6 8 7 2
1 5 8 2 6 7 9 3 4
3 6 7 4 5 9 2 1 8
9 4 2 3 8 1 6 5 7



In [65]:
board="""_74___35_
__1__5___
__2__4__1
1___8__7_
___6_2___
_6__1___3
9__2__1__
___9__6__
_37___42_"""
s = Sudoku(3,3,board)
print s

_ 7 4 _ _ _ 3 5 _
_ _ 1 _ _ 5 _ _ _
_ _ 2 _ _ 4 _ _ 1
1 _ _ _ 8 _ _ 7 _
_ _ _ 6 _ 2 _ _ _
_ 6 _ _ 1 _ _ _ 3
9 _ _ 2 _ _ 1 _ _
_ _ _ 9 _ _ 6 _ _
_ 3 7 _ _ _ 4 2 _


In [66]:
d = DLXTechnique(s)
for ss in d.solutions():
    print ss
    print

6 7 4 8 9 1 3 5 2
3 9 1 7 2 5 8 6 4
5 8 2 3 6 4 7 9 1
1 2 9 4 8 3 5 7 6
4 5 3 6 7 2 9 1 8
7 6 8 5 1 9 2 4 3
9 4 6 2 3 7 1 8 5
2 1 5 9 4 8 6 3 7
8 3 7 1 5 6 4 2 9

